# Cross Validation
As time-series have the inherent structure we could run into problems with traditional shuffled Kfolds cross-validation. hcrystalball implements forward rolling cross-validation making training set consist only of observations that occurred prior to the observations that form the test set.

<center><img src="https://raw.githubusercontent.com/heidelbergcement/hcrystalball/master/docs/_static/FinerTimeSplit.svg"></center>

In [ ]:
from hcrystalball.model_selection import FinerTimeSplit
from sklearn.model_selection import cross_validate
from hcrystalball.wrappers import ExponentialSmoothingWrapper

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from hcrystalball.utils import get_sales_data

df = get_sales_data(n_dates=100, 
                    n_assortments=1,
                    n_states=1, 
                    n_stores=1)
X, y = pd.DataFrame(index=df.index), df['Sales']

## Native Cross Validation

In [ ]:
cross_validate(ExponentialSmoothingWrapper(), 
               X, 
               y, 
               cv=FinerTimeSplit(horizon=5, n_splits=2), 
               scoring='neg_mean_absolute_error')

## Grid search and model selection

Model selection and parameter tuning is the area where hcrystalball really shines. There is ongoing and probably a never-ending discussion about superiority or inferiority of ML techniques over common statistical/econometrical ones. Why not try both? The problem of a simple comparison between the performance of different kind of algorithms such as SARIMAX, Prophet, regularized linear models, and XGBoost lead to hcrystalball. Let's see how to do it!

In [ ]:
from hcrystalball.compose import TSColumnTransformer
from hcrystalball.feature_extraction import SeasonalityTransformer
from hcrystalball.wrappers import ProphetWrapper
from hcrystalball.wrappers import get_sklearn_wrapper
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

from hcrystalball.wrappers import SarimaxWrapper
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd

### Define our pipeline

In [ ]:
sklearn_model_pipeline = Pipeline([
    ('seasonality', SeasonalityTransformer(freq='D')),
    ('model', 'passthrough') # this will be overwritten by param grid
]) 

### Define pipeline parameters including different models

In [ ]:
param_grid = [{'model': [sklearn_model_pipeline],
               'model__model':[get_sklearn_wrapper(RandomForestRegressor, random_state=42), 
                               get_sklearn_wrapper(LinearRegression)]},
              {'model': [ProphetWrapper()],
               'model__seasonality_mode':['multiplicative', 'additive']},
              {'model': [SarimaxWrapper(order=(2,1,1), suppress_warnings=True)]}
             ]

### Run native grid search

In [ ]:
grid_search = GridSearchCV(estimator=sklearn_model_pipeline, 
                           param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', 
                           cv=FinerTimeSplit(horizon=5, n_splits=2),
                           refit=False, 
                           error_score=np.nan) 

In [ ]:
results = grid_search.fit(X, y)

In [ ]:
pd.DataFrame(results.cv_results_).sort_values('rank_test_score').head()

It seems that the best model is sklearn with RandomForestRegressor, but in time-series, it is often also a good idea to check how the forecasts look like. Unfortunately, this is not possible with sklearn. grid_search is returning just the results, not the predictions of individual models for different splits. hcrystalball thus implements special scoring functions that track all data from the grid_search.

## Custom scorer

In [ ]:
from hcrystalball.metrics import make_ts_scorer
from sklearn.metrics import mean_absolute_error

In [ ]:
scoring = make_ts_scorer(mean_absolute_error, 
                         greater_is_better=False)

In [ ]:
grid_search = GridSearchCV(estimator=sklearn_model_pipeline, 
                           param_grid=param_grid, 
                           scoring=scoring, 
                           cv=FinerTimeSplit(horizon=5, n_splits=2),
                           refit=False, 
                           error_score=np.nan) 
results = grid_search.fit(X, y)

In [ ]:
results.scorer_.cv_data.loc[:,lambda x: x.columns != 'split'].plot();

hcrystalball internally tracks data based on unique model hashes since model string represantations (reprs) are very long for usable columns names in dataframe, but if you are curious i.e. what was the worse model not to use it for further experiment, you can do it with scorers `estimator_ids` attribute

In [ ]:
results.scorer_.cv_data.head()

We can get to the model definitions using hash in `results.scorer_.estimator_ids` dict